In [0]:
# !unzip -uq "/content/drive/My Drive/roi.zip" -d "/content/drive/My Drive/roi_sample"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
path_plus='roi_sample/E01' # 무표정
path_zero='roi_sample/E02' # 웃음
path_minus='roi_sample/E03' # 찡그림
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plus 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

zero 경로에 저장된 파일의 개수:2880
plus 경로에 저장된 파일의 개수:2880
minus 경로에 저장된 파일의 개수:2880


In [3]:
xsize=100
ysize=100

In [4]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [5]:
read_dir(path_zero, 0) #'plus emotion')
read_dir(path_plus, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [6]:
x=np.array(x)
y=np.array(y)

In [7]:
print(x.shape)

(8640, 100, 100, 3)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

input_shape = (xsize,ysize, 3)

batch_size = 32
num_classes = 2
epochs = 15

In [9]:
print(x_test.shape)
print(y_test.shape)

(864, 100, 100, 3)
(864,)


In [10]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.applications import ResNet50, ResNet101 

input_shape = (100, 100, 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable=False

x = Flatten()(base_model.output)
output = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# model.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  # This is added back by InteractiveShellApp.init_path()


In [12]:
model.fit(x_train, y_train, validation_split=0.2, 
          epochs=30, batch_size=100, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)


Train on 6220 samples, validate on 1556 samples
Epoch 1/30
6220/6220 [==============================] - 36s 6ms/step - loss: 1.2124 - accuracy: 0.7870 - val_loss: 15612.2429 - val_accuracy: 0.3560
Epoch 2/30
6220/6220 [==============================] - 20s 3ms/step - loss: 0.0661 - accuracy: 0.9744 - val_loss: 1.1681 - val_accuracy: 0.3445
Epoch 3/30
6220/6220 [==============================] - 20s 3ms/step - loss: 0.0260 - accuracy: 0.9912 - val_loss: 1.1254 - val_accuracy: 0.2995
Epoch 4/30
6220/6220 [==============================] - 21s 3ms/step - loss: 0.0157 - accuracy: 0.9942 - val_loss: 1.1009 - val_accuracy: 0.2995
Epoch 5/30
6220/6220 [==============================] - 21s 3ms/step - loss: 0.0260 - accuracy: 0.9915 - val_loss: 1.0997 - val_accuracy: 0.3066
Epoch 6/30
6220/6220 [==============================] - 21s 3ms/step - loss: 0.0249 - accuracy: 0.9931 - val_loss: 1.1000 - val_accuracy: 0.3014
Epoch 7/30
6220/6220 [==============================] - 21s 3ms/step - loss: 

In [ ]:
model.save('save/ResNet_freezing(full).h5')
print('모델 저장이 완료되었습니다')

In [13]:
test_path = "test_dataset/Happy/Happy" # 기쁨만 모인 test set
test_path2 = "test_dataset/Neutral/Neutral" # 무표정만 모인 test set 

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [14]:
# test set 기쁨
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

In [16]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [15]:
# 예측(test set 기쁨)
co=0
for i in range(200):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n',co/200.)


00020201000000000000002000000000000200000000200020000000000000002000000002000000200000001020200000002000000200002002000000020000100000000000022020000020000000000020020222002000000000000000000200000000
 0.015


In [17]:
# 예측(test set 무표정)
co=0
for i in range(200):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n',co/200.)

00020201000000000000002000000000000200000000200020000000000000002000000002000000200000001020200000002000000200002002000000020000100000000000022020000020000000000020020222002000000000000000000200000000
 0.85


In [0]:
# 기쁨 예측 정확도 : 1.5 %
# 무표정 예측 정확도 : 85 %

# 역시 무표정 예측 정확도가 더 높음 
# 데이터 증식이나 모델 파라미터 조정이 필요함(50 % 이상 목표)